In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.environ['LD_LIBRARY_PATH'] = "$LD_LIBRARY_PATH:/usr/local/cuda-11.3/lib64/"
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork, Trajectory
#import torch
#import torch.nn as nn
#from torch.utils.data import DataLoader
#import pandas as pd
import networkx as nx
import osmnx as ox
from models.hrnr_original.utils import *
from models.hrnr_original.conf import beijing_hparams
# from models.hrnr_original.train import train_struct_cmt, train_fnc_cmt_rst
from models.hrnr_original.model import *
import torch
import torch.nn.functional as F
from torch import optim
from models.training.hrnr_data_generation import *
from models import HRNRModel

In [2]:
torch.cuda.set_device(1)

In [ ]:
# try on original
hparams = dict_to_object(beijing_hparams)
hparams.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
adj, features, sp_label = load_gae_data(hparams) 

In [ ]:
print(len(np.unique(features[:, 2])))

In [ ]:
train_struct_cmt()

In [ ]:
train_fnc_cmt_rst()

In [20]:
# load road network
network = RoadNetwork()
network.load("../../osm_data/porto")
data = network.generate_road_segment_pyg_dataset()


In [ ]:
trajectory = Trajectory(
    "../../datasets/trajectories/Porto/road_segment_map_final.csv", nrows=10000000
)

In [ ]:
adjp, featuresp, sp_labelp, t_adjp = get_data(network, trajectory)

In [ ]:
# train structural region and functional region matrices
train_struct_cmt_custom(adjp, featuresp, sp_labelp)
train_fnc_cmt_rst_custom(adjp, featuresp, sp_labelp, t_adjp)

In [26]:
# Build model
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = HRNRModel(data, device=device, network=network, data_path="hrnr_data")

In [27]:
model.train()

/home/pheinemeyer/miniconda3/envs/road/lib/python3.9/site-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 50, avg_loss: 5.587236075401306
Epoch: 100, avg_loss: 4.113418843746185
Epoch: 150, avg_loss: 3.5936533387502037
Epoch: 200, avg_loss: 3.276423048377037
Epoch: 250, avg_loss: 3.005496003627777
Epoch: 300, avg_loss: 2.8228054213523865
Epoch: 350, avg_loss: 2.6892074278422764
Epoch: 400, avg_loss: 2.5974207600951194
Epoch: 450, avg_loss: 2.5260202746921117
Epoch: 500, avg_loss: 2.4614560291767122
Epoch: 550, avg_loss: 2.4080251513827933
Epoch: 600, avg_loss: 2.3620931841929753
Epoch: 650, avg_loss: 2.337940115745251
Epoch: 700, avg_loss: 2.3058624608176097
Epoch: 750, avg_loss: 2.2762662822405497
Epoch: 800, avg_loss: 2.250242542475462
Epoch: 850, avg_loss: 2.2266648065342625
Epoch: 900, avg_loss: 2.2051778309875063
Epoch: 950, avg_loss: 2.186329327257056


In [28]:
model.save_model("../model_states/hrnr")

In [29]:
z = model.load_emb()

/home/pheinemeyer/miniconda3/envs/road/lib/python3.9/site-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [30]:
print(z.shape)

(11331, 128)
